In [ ]:
from ultralytics import YOLO
import cv2
import mediapipe as mp
import numpy as np
import cv2
import pickle
from PIL import Image
import numpy as np

# Load the trained model
with open('exercise_nn_model_lat.pkl', 'rb') as f:
    modelnn = pickle.load(f)

class Result:
    def __init__(self, names):
        self.names = names

def calculate_angle_between_three_points_2d(pointA, pointB, pointC):

    # Convert points to numpy arrays
    A = np.array(pointA)
    B = np.array(pointB)
    C = np.array(pointC)
    
    # Create vectors AB and BC
    AB = A - B
    BC = C - B
    
    # Normalize the vectors
    AB_norm = np.linalg.norm(AB)
    BC_norm = np.linalg.norm(BC)
    
    # Ensure the vectors are not zero-length to avoid division by zero
    if AB_norm == 0 or BC_norm == 0:
        return None
    
    AB_normalized = AB / AB_norm
    BC_normalized = BC / BC_norm
    
    # Calculate the dot product of the normalized vectors
    dot_product = np.dot(AB_normalized, BC_normalized)
    
    # Clip dot_product to avoid possible numerical errors
    dot_product = np.clip(dot_product, -1.0, 1.0)
    
    # Calculate the angle in radians and then convert to degrees
    angle_radians = np.arccos(dot_product)
    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point

    # Calculate the vectors
    ba = a - b
    bc = c - b

    # Calculate the cosine of the angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    # Convert the angle from radians to degrees
    return np.degrees(angle)

def calculate_midpoint(coord1, coord2):
    x = (coord1[0] + coord2[0]) / 2
    y = (coord1[1] + coord2[1]) / 2
    z = (coord1[2] + coord2[2]) / 2
    return np.array([x, y, z])

def calculate_angle_using_cross_product(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point

    # Calculate the vectors
    ba = a - b
    bc = c - b

    # Compute the cross product of the vectors
    cross_product = np.cross(ba, bc)

    # Compute the dot product
    dot_product = np.dot(ba, bc)

    # Calculate the angle using the cross and dot products
    angle = np.arctan2(np.linalg.norm(cross_product), dot_product)

    # Convert the angle from radians to degrees
    return np.degrees(angle)

# Function to calculate the distance between two points
def calculate_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a - b)

# Load the model
model = YOLO('model_final.pt')

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3, min_tracking_confidence=0.3)

# Open the video file
video_path = 'bier_60.mp4'

cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video details
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
window_name = 'Frame'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 720,750 )  # Resize the window to 800x600

frame_count=0
ex_elbo_dis = [0.0, 0.0, 0.0]

lat_max = []
squat_angle = 1.0
squat_stand = 0.0
elbow_angle_temp =0.0
puttext =0
error_el = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    # Perform inference on the current frame
    results = model(frame)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame for pose detection
    pose_results = pose.process(frame_rgb)

    names = results[0].names
    top1 = results[0].probs.top1
    if top1 == 0:
        classname = names[0]    # bicep
    if top1 == 1:
        classname = names[1]    # dumbbell row 
    if top1 == 2:
        classname = names[2]    # incline dumbbell press
    if top1 == 3:
        classname = names[3]    # lat pull down
    if top1 == 4:
        classname = names[4]    # squat
        
    cv2.putText(frame, f'Class: {classname}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    if puttext != 0 and error_el != 0:
        cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        puttext -= 1
        if puttext == 0:
            error_el = 0
            
    if pose_results.pose_landmarks:
        # Draw the skeleton on the frame
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        # Extract landmark positionst_ma
        landmarks = pose_results.pose_landmarks.landmark


        landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks])
        # Flatten landmarks for model input
        pose_data = landmarks_array.flatten()
        
        # Reshape data to match model's input format
        pose_data = pose_data.reshape(1, -1)
        
        # Predict the exercise class
        predicted_class = modelnn.predict(pose_data)[0]

        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        
        right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
        
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
        
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

        right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
        left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]

        if classname == names[0]: # bicep
            if left_shoulder.visibility > 0.5:
                h, w, _ = frame.shape
                midpoint_pixel = (int(left_shoulder.x * w), int(left_shoulder.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                
                angle = calculate_angle(
                [left_shoulder.x, left_shoulder.y, left_shoulder.z],
                [left_elbow.x, left_elbow.y, left_elbow.z],
                [left_wrist.x, left_wrist.y, left_wrist.z]
                )
                cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                if frame_count > 3:
                    elbow_angle = calculate_angle([left_shoulder.x, left_shoulder.y, left_shoulder.z],
                    [left_elbow.x, left_elbow.y, left_elbow.z],
                    [left_hip.x, left_hip.y, left_hip.z])
    
                    if abs(elbow_angle-elbow_angle_temp)> 35 :
                        error_el = 1
                        puttext = 15 # 1 sec
                        cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                if frame_count % 3 == 0: # 2 sec
                    elbow_angle_temp = calculate_angle([left_shoulder.x, left_shoulder.y, left_shoulder.z],
                    [left_elbow.x, left_elbow.y, left_elbow.z],
                    [left_hip.x, left_hip.y, left_hip.z])
            else:
                h, w, _ = frame.shape
                midpoint_pixel = (int(right_shoulder.x * w), int(right_shoulder.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                
                angle = calculate_angle(
                [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [right_elbow.x, right_elbow.y, right_elbow.z],
                [right_wrist.x, right_wrist.y, right_wrist.z]
                )
                cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                if frame_count > 3:
                    elbow_angle = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [right_elbow.x, right_elbow.y, right_elbow.z],
                    [right_hip.x, right_hip.y, right_hip.z])
    
                    if abs(elbow_angle-elbow_angle_temp)> 35 :
                        error_el = 1
                        puttext = 15 # 1 sec
                        cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                if frame_count % 3 == 0: # 2 sec
                    elbow_angle_temp = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [right_elbow.x, right_elbow.y, right_elbow.z],
                    [right_hip.x, right_hip.y, right_hip.z])

        elif classname == names[1]: # dumbbell row
            leftangle = calculate_angle(
            [left_shoulder.x, left_shoulder.y, left_shoulder.z],
            [left_elbow.x, left_elbow.y, left_elbow.z],
            [left_wrist.x, left_wrist.y, left_wrist.z]
                )
            rightangle = calculate_angle(
            [right_shoulder.x, right_shoulder.y, right_shoulder.z],
            [right_elbow.x, right_elbow.y, right_elbow.z],
            [right_wrist.x, right_wrist.y, right_wrist.z]
                )
             
            if leftangle > rightangle:
                back_mid = calculate_midpoint([left_shoulder.x, left_shoulder.y, left_shoulder.z],[left_hip.x,left_hip.y,left_hip.z])
    
                if left_elbow.y*1.05 < back_mid[1]:
                    cv2.putText(frame, f'Elbow down', (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
                if leftangle < np.degrees(1.5 ) : # 100 degree
                    cv2.putText(frame, f'keep left arm straight', (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else :
                back_mid = calculate_midpoint([right_shoulder.x, right_shoulder.y, right_shoulder.z],[right_hip.x,right_hip.y,right_hip.z])
                if right_elbow.y *1.05< back_mid[1]:
                    cv2.putText(frame, f'Elbow down', (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
                if rightangle < np.degrees(1.5 ) : 
                    cv2.putText(frame, f'keep right arm straight', (10, 290), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)                            

        elif classname == names[2]:  # Incline Dumbbell Press
            angle = calculate_angle(
            [left_shoulder.x, left_shoulder.y, left_shoulder.z],
            [left_elbow.x, left_elbow.y, left_elbow.z],
            [left_wrist.x, left_wrist.y, left_wrist.z]
            )
            cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            h, w, _ = frame.shape
            midpoint_pixel = (int(right_shoulder.x * w), int(right_shoulder.y * h))
            cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
            midpoint_pixel = (int(right_elbow.x * w), int(right_elbow.y * h))
            cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
            midpoint_pixel = (int(right_hip.x * w), int(right_hip.y * h))
            cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
            
            if left_elbow.y*0.95 > left_shoulder.y:
                shoulder_angle = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [right_elbow.x, right_elbow.y, right_elbow.z],
                [right_hip.x, right_hip.y, right_hip.z])
                cv2.putText(frame, f'shoul_ang: {int(shoulder_angle)}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

                if shoulder_angle > np.degrees(2.7925268):
                    cv2.putText(frame, f'too wide elbow', (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
        elif classname == names[3]:  # Lat Pull Down

            shoulder_distance = calculate_distance(
                [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [left_shoulder.x, left_shoulder.y, left_shoulder.z]
            )
            wrist_distance = calculate_distance(
                [right_wrist.x, right_wrist.y, right_wrist.z],
                [left_wrist.x, left_wrist.y, left_wrist.z]
            )

            if wrist_distance > shoulder_distance * 2.2:
                cv2.putText(frame, "Keep Wrists Closer", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
        elif classname == names[4]:  # Squat (already implemented)
            error_sh_kne = 0
            error_posi = 0
            
            shoulder_distance = calculate_distance(
                [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [left_shoulder.x, left_shoulder.y, left_shoulder.z]
            )
            knee_distance = calculate_distance(
                [right_knee.x, right_knee.y, right_knee.z],
                [left_knee.x, left_knee.y, left_knee.z]
            )
            ankle_distance = calculate_distance(
                [right_ankle.x, right_ankle.y, right_ankle.z],
                [left_ankle.x, left_ankle.y, left_ankle.z]
            )
            
            squat_angle = calculate_angle(
            [right_hip.x, right_hip.y, right_hip.z],
            [right_knee.x, right_knee.y, right_knee.z],
            [right_ankle.x, right_ankle.y, right_ankle.z]
            ) 
            #flip =0
            if right_knee.visibility > 0.5:
                if squat_angle > np.degrees(2.7925268 ) :
                    squat_stand = abs(right_knee.x-right_foot_index.x)
                    if ankle_distance > (shoulder_distance * 1.43) :
                        error_sh_kne = 1
                        cv2.putText(frame, f'adjust your ankle closer', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        
                    if ankle_distance < (shoulder_distance * 0.8):
                        error_sh_kne = 1
                        cv2.putText(frame, f'adjust your ankle wider', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                if squat_stand !=0:
                    if right_knee.x < right_foot_index.x :
                        if right_knee.x + squat_stand < right_foot_index.x:
                            cv2.putText(frame, f'false position', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            else:
                if squat_angle > np.degrees(2.7925268 ) :
                    squat_stand = abs(left_knee.x-left_foot_index.x)
                    if ankle_distance > (shoulder_distance * 1.43) :
                        error_sh_kne = 1
                        cv2.putText(frame, f'adjust your ankle closer', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    if ankle_distance < (shoulder_distance * 0.8):
                        error_sh_kne = 1
                        cv2.putText(frame, f'adjust your ankle wider', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        
                if squat_stand !=0:
                    if left_knee.x < left_foot_index.x :

                        if left_knee.x + squat_stand < left_foot_index.x:
                            cv2.putText(frame, f'false position', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when job is finished
cap.release()
# out.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
import cv2
import mediapipe as mp
import numpy as np
import cv2
import pickle
from PIL import Image
import numpy as np

# Load the trained model
with open('exercise_nn_model_bi.pkl', 'rb') as f:
    modelnn = pickle.load(f)



class Result:
    def __init__(self, names):
        self.names = names

def calculate_angle_with_vertical(point1, point2):
    # Convert points to numpy arrays
    p1 = np.array(point1)
    p2 = np.array(point2)
    
    # Calculate the vector from point1 to point2
    vector = p2 - p1
    
    # Define the vertical z-axis unit vector
    z_axis = np.array([0, 0, 1])
    
    # Calculate the dot product between the vector and the z-axis
    dot_product = np.dot(vector, z_axis)
    
    # Calculate the magnitudes of the vector and the z-axis
    vector_magnitude = np.linalg.norm(vector)
    z_axis_magnitude = np.linalg.norm(z_axis)
    
    # Calculate the cosine of the angle using the dot product formula
    cos_angle = dot_product / (vector_magnitude * z_axis_magnitude)
    
    # Ensure the cosine value is within the valid range for arccos
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    
    # Calculate the angle in radians
    angle_rad = np.arccos(cos_angle)
    
    # Convert the angle to degrees
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg
    
def calculate_angle_between_three_points_2d(pointA, pointB, pointC):
    """
    Calculate the angle between three points in 2D space.
    
    Args:
    - pointA, pointB, pointC: Lists or arrays with the coordinates [x, y].
    
    Returns:
    - Angle in degrees between the vectors AB and BC.
    """
    # Convert points to numpy arrays
    A = np.array(pointA)
    B = np.array(pointB)
    C = np.array(pointC)
    
    # Create vectors AB and BC
    AB = A - B
    BC = C - B
    
    # Normalize the vectors
    AB_norm = np.linalg.norm(AB)
    BC_norm = np.linalg.norm(BC)
    
    # Ensure the vectors are not zero-length to avoid division by zero
    if AB_norm == 0 or BC_norm == 0:
        return None
    
    AB_normalized = AB / AB_norm
    BC_normalized = BC / BC_norm
    
    # Calculate the dot product of the normalized vectors
    dot_product = np.dot(AB_normalized, BC_normalized)
    
    # Clip dot_product to avoid possible numerical errors
    dot_product = np.clip(dot_product, -1.0, 1.0)
    
    # Calculate the angle in radians and then convert to degrees
    angle_radians = np.arccos(dot_product)
    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point

    # Calculate the vectors
    ba = a - b
    bc = c - b

    # Calculate the cosine of the angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    # Convert the angle from radians to degrees
    return np.degrees(angle)

def calculate_angle_using_cross_product(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point

    # Calculate the vectors
    ba = a - b
    bc = c - b

    # Compute the cross product of the vectors
    cross_product = np.cross(ba, bc)

    # Compute the dot product
    dot_product = np.dot(ba, bc)

    # Calculate the angle using the cross and dot products
    angle = np.arctan2(np.linalg.norm(cross_product), dot_product)

    # Convert the angle from radians to degrees
    return np.degrees(angle)

def calculate_angle_temp(vec1, vec2):
    unit_vec1 = vec1 / np.linalg.norm(vec1)
    unit_vec2 = vec2 / np.linalg.norm(vec2)
    dot_product = np.dot(unit_vec1, unit_vec2)
    angle = np.arccos(dot_product)  # Angle in radians
    return np.degrees(angle)  # Convert to degrees


# Function to calculate the distance between two points
def calculate_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a - b)

def calculate_midpoint(coord1, coord2):
    x = (coord1[0] + coord2[0]) / 2
    y = (coord1[1] + coord2[1]) / 2
    z = (coord1[2] + coord2[2]) / 2
    return np.array([x, y, z])
# Load the model
model = YOLO('model_final.pt')

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3, min_tracking_confidence=0.3)

# Open the video file
video_path = 'bier_60.mp4'

cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video details
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
window_name = 'Frame'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 720,750 )  # Resize the window to 800x600
# output_path = 'output_video.avi'
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_count=0
ex_elbo_dis = [0.0, 0.0, 0.0]

lat_max = []
squat_angle = 1.0
squat_stand = 0.0
elbow_angle_temp =0.0
puttext =0
error_el = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    # Perform inference on the current frame
    results = model(frame)
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    
    # Process the frame for pose detection
    pose_results = pose.process(frame_rgb)

    
    names = results[0].names
    top1 = results[0].probs.top1
    if top1 == 0:
        classname = names[0]    # bicep
    if top1 == 1:
        classname = names[1]    # dumbbell row 
    if top1 == 2:
        classname = names[2]    # incline dumbbell press
    if top1 == 3:
        classname = names[3]    # lat pull down
    if top1 == 4:
        classname = names[4]    # squat
        
    '''cv2.putText(frame, f'Class: {classname}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)'''
    if puttext != 0 and error_el != 0:
        cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        puttext -= 1
        if puttext ==0:
            error_el = 0
            
    if pose_results.pose_landmarks:
        # Draw the skeleton on the frame
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        # Extract landmark positionst_ma
        landmarks = pose_results.pose_landmarks.landmark
        
        landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks])
                # Flatten landmarks for model input
        pose_data = landmarks_array.flatten()
        
                # Reshape data to match model's input format
        pose_data = pose_data.reshape(1, -1)
        
                # Predict the exercise class
        predicted_class = modelnn.predict(pose_data)[0]
        predicted_class_str = str(predicted_class).strip()
        
                # Display the predicted class on the screen
        cv2.putText(frame, f'Exercise: {predicted_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        
        right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
        
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
        
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

        right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
        left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]

        
        if predicted_class_str == 'bicep':
            if left_shoulder.visibility > 0.5:
                h, w, _ = frame.shape
                midpoint_pixel = (int(left_shoulder.x * w), int(left_shoulder.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                
                angle = calculate_angle(
                [left_shoulder.x, left_shoulder.y, left_shoulder.z],
                [left_elbow.x, left_elbow.y, left_elbow.z],
                [left_wrist.x, left_wrist.y, left_wrist.z]
                )
                cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                if frame_count > 3:
                    elbow_angle = calculate_angle([left_shoulder.x, left_shoulder.y, left_shoulder.z],
                    [left_elbow.x, left_elbow.y, left_elbow.z],
                    [left_hip.x, left_hip.y, left_hip.z])
    
                    if abs(elbow_angle-elbow_angle_temp)> 35 :
                        error_el = 1
                        puttext = 15 # 1 sec
                        cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                if frame_count % 3 == 0: # 2 sec
                    elbow_angle_temp = calculate_angle([left_shoulder.x, left_shoulder.y, left_shoulder.z],
                    [left_elbow.x, left_elbow.y, left_elbow.z],
                    [left_hip.x, left_hip.y, left_hip.z])
            else:
                h, w, _ = frame.shape
                midpoint_pixel = (int(right_shoulder.x * w), int(right_shoulder.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                
                angle = calculate_angle(
                [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [right_elbow.x, right_elbow.y, right_elbow.z],
                [right_wrist.x, right_wrist.y, right_wrist.z]
                )
                cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                if frame_count > 3:
                    elbow_angle = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [right_elbow.x, right_elbow.y, right_elbow.z],
                    [right_hip.x, right_hip.y, right_hip.z])
    
                    if abs(elbow_angle-elbow_angle_temp)> 35 :
                        error_el = 1
                        puttext = 15 # 1 sec
                        cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                if frame_count % 3 == 0: # 2 sec
                    elbow_angle_temp = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [right_elbow.x, right_elbow.y, right_elbow.z],
                    [right_hip.x, right_hip.y, right_hip.z])
            
        else :
            
            if classname == names[0]: # bicep
                if left_shoulder.visibility > 0.5:
                    h, w, _ = frame.shape
                    midpoint_pixel = (int(left_shoulder.x * w), int(left_shoulder.y * h))
                    cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                    
                    angle = calculate_angle(
                    [left_shoulder.x, left_shoulder.y, left_shoulder.z],
                    [left_elbow.x, left_elbow.y, left_elbow.z],
                    [left_wrist.x, left_wrist.y, left_wrist.z]
                    )
                    cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    
                    if frame_count > 3:
                        elbow_angle = calculate_angle([left_shoulder.x, left_shoulder.y, left_shoulder.z],
                        [left_elbow.x, left_elbow.y, left_elbow.z],
                        [left_hip.x, left_hip.y, left_hip.z])
        
                        if abs(elbow_angle-elbow_angle_temp)> 40 :
                            error_el = 1
                            puttext = 15 # 1 sec
                            cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    if frame_count % 3 == 0: # 2 sec
                        elbow_angle_temp = calculate_angle([left_shoulder.x, left_shoulder.y, left_shoulder.z],
                        [left_elbow.x, left_elbow.y, left_elbow.z],
                        [left_hip.x, left_hip.y, left_hip.z])
                else:
                    h, w, _ = frame.shape
                    midpoint_pixel = (int(right_shoulder.x * w), int(right_shoulder.y * h))
                    cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                    
                    angle = calculate_angle(
                    [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [right_elbow.x, right_elbow.y, right_elbow.z],
                    [right_wrist.x, right_wrist.y, right_wrist.z]
                    )
                    cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    
                    if frame_count > 3:
                        elbow_angle = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                        [right_elbow.x, right_elbow.y, right_elbow.z],
                        [right_hip.x, right_hip.y, right_hip.z])
        
                        if abs(elbow_angle-elbow_angle_temp)> 40 :
                            error_el = 1
                            puttext = 15 # 1 sec
                            cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    if frame_count % 3 == 0: # 2 sec
                        elbow_angle_temp = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                        [right_elbow.x, right_elbow.y, right_elbow.z],
                        [right_hip.x, right_hip.y, right_hip.z])


    
            elif classname == names[1]: # dumbbell row
                leftangle = calculate_angle(
                [left_shoulder.x, left_shoulder.y, left_shoulder.z],
                [left_elbow.x, left_elbow.y, left_elbow.z],
                [left_wrist.x, left_wrist.y, left_wrist.z]
                    )
                rightangle = calculate_angle(
                [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [right_elbow.x, right_elbow.y, right_elbow.z],
                [right_wrist.x, right_wrist.y, right_wrist.z]
                    )
                #h, w, _ = frame.shape
                #midpoint_pixel = (int(right_shoulder.x * w), int(right_shoulder.y * h))
                #cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                cv2.putText(frame, f'leftangle: {int(leftangle)}', (10, 230), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                cv2.putText(frame, f'rightangle: {int(rightangle)}', (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                 
                if leftangle > rightangle:
                    cv2.putText(frame, f'useleft', (10, 280), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    back_mid = calculate_midpoint([left_shoulder.x, left_shoulder.y, left_shoulder.z],[left_hip.x,left_hip.y,left_hip.z])
                    h, w, _ = frame.shape
                    midpoint_pixel = (int(back_mid[0] * w), int(back_mid[1] * h))
                    cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
        
        
                    print(f"elbow/midcakV {left_elbow.x*0.92},{back_mid[0]}")
                    if left_elbow.x*0.91 > back_mid[0]:
                        cv2.putText(frame, f'Elbow down', (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        
                    if leftangle < np.degrees(1.5 ) : # 100 degree
                        #print(f"leftangle: {leftangle}")
                        cv2.putText(frame, f'keep left arm straight', (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else :
                    back_mid = calculate_midpoint([right_shoulder.x, right_shoulder.y, right_shoulder.z],[right_hip.x,right_hip.y,right_hip.z])
                    if right_elbow.x*0.91 > back_mid[0]:
                        print(f"elbow/midcakV {right_elbow.x*0.9},{back_mid[0]}")
                        cv2.putText(frame, f'Elbow down', (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        
                    if rightangle < np.degrees(1.5 ) : 
                        #print(f"rightangle: {rightangle}")
                        cv2.putText(frame, f'keep right arm straight', (10, 290), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)                    

            
            elif classname == names[2]:  # Incline Dumbbell Press
                angle = calculate_angle_between_three_points_2d(
                [left_elbow.x, left_elbow.y],
                [left_shoulder.x, left_shoulder.y],
                [left_hip.x, left_hip.y]
                )
                angle2 = calculate_angle_between_three_points_2d(
                [right_elbow.x, right_elbow.y],
                [right_shoulder.x, right_shoulder.y],
                [right_hip.x, right_hip.y]
                )
                
                
                #cv2.putText(frame, f'222Elbow Angle: {int(angle2)}', (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                h, w, _ = frame.shape
                
                midpoint_pixel = (int(left_shoulder.x * w), int(left_shoulder.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                midpoint_pixel = (int(left_elbow.x * w), int(left_elbow.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                midpoint_pixel = (int(left_hip.x * w), int(left_hip.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
    
                #if  angle2< np.degrees(0.78539816 ):#camera 30
                    #angle -= 30
                cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                if angle > np.degrees(1.3962634) and angle < np.degrees(1.57079633):#80 de
                    cv2.putText(frame, f'too wide elbow', (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            elif classname == names[3]:  # Lat Pull Down

                shoulder_distance = calculate_distance(
                    [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [left_shoulder.x, left_shoulder.y, left_shoulder.z]
                )
                wrist_distance = calculate_distance(
                    [right_wrist.x, right_wrist.y, right_wrist.z],
                    [left_wrist.x, left_wrist.y, left_wrist.z]
                )
                cv2.putText(frame, f'Wrist Distance: {wrist_distance:.2f}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                cv2.putText(frame, f'shoul Distance: { shoulder_distance * 2.2:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
                if wrist_distance > shoulder_distance * 2.2:
                    cv2.putText(frame, "Keep Wrists Closer", (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
    
            elif classname == names[4]:  # Squat (already implemented)
                error_sh_kne = 0
                error_posi = 0
                
                shoulder_distance = calculate_distance(
                    [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [left_shoulder.x, left_shoulder.y, left_shoulder.z]
                )
                knee_distance = calculate_distance(
                    [right_knee.x, right_knee.y, right_knee.z],
                    [left_knee.x, left_knee.y, left_knee.z]
                )
                ankle_distance = calculate_distance(
                    [right_ankle.x, right_ankle.y, right_ankle.z],
                    [left_ankle.x, left_ankle.y, left_ankle.z]
                )
                
                squat_angle = calculate_angle(
                [right_hip.x, right_hip.y, right_hip.z],
                [right_knee.x, right_knee.y, right_knee.z],
                [right_ankle.x, right_ankle.y, right_ankle.z]
                ) 
                #flip =0
                if right_knee.visibility > 0.5:
                    if squat_angle > np.degrees(2.7925268 ) :
                        squat_stand = abs(right_knee.x-right_foot_index.x)
                        if ankle_distance > (shoulder_distance * 1.43) :
                            error_sh_kne = 1
                            cv2.putText(frame, f'adjust your ankle closer', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            
                        if ankle_distance < (shoulder_distance * 0.8):
                            error_sh_kne = 1
                            cv2.putText(frame, f'adjust your ankle wider', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    #else:
                        #if right_knee.x > right_foot_index.x :
                           #flip=1
                        
        
                    if squat_stand !=0:
                        if right_knee.x < right_foot_index.x :
                            if right_knee.x + (squat_stand) < right_foot_index.x:
                                cv2.putText(frame, f'false position', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        #if flip== 1:
                            #if right_knee.x - (squat_stand) > right_foot_index.x:
                                #cv2.putText(frame, f'false position xxxx', (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    if squat_angle > np.degrees(2.7925268 ) :
                        squat_stand = abs(left_knee.x-left_foot_index.x)
                        if ankle_distance > (shoulder_distance * 1.43) :
                            error_sh_kne = 1
                            cv2.putText(frame, f'adjust your ankle closer', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
                        if ankle_distance < (shoulder_distance * 0.8):
                            error_sh_kne = 1
                            cv2.putText(frame, f'adjust your ankle wider', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2) 
                    #else:
                        #if left_knee.x > left_foot_index.x :
                            #flip=1
                            
                    if squat_stand !=0:
                        if left_knee.x < left_foot_index.x :
        
                            if left_knee.x + (squat_stand) < left_foot_index.x:
                                cv2.putText(frame, f'false position', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)



    
    # Display the frame
    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when job is finished
cap.release()
# out.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
import cv2
import mediapipe as mp
import numpy as np
import cv2
import pickle
from PIL import Image
import numpy as np


import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import joblib
# Load the trained model
#with open('exercise_nn_model_lat.pkl', 'rb') as f:
    #modelnn = pickle.load(f)



class Result:
    def __init__(self, names):
        self.names = names


def calculate_angle_between_three_points_2d(pointA, pointB, pointC):

    # Convert points to numpy arrays
    A = np.array(pointA)
    B = np.array(pointB)
    C = np.array(pointC)
    
    # Create vectors AB and BC
    AB = A - B
    BC = C - B
    
    # Normalize the vectors
    AB_norm = np.linalg.norm(AB)
    BC_norm = np.linalg.norm(BC)
    
    # Ensure the vectors are not zero-length to avoid division by zero
    if AB_norm == 0 or BC_norm == 0:
        return None
    
    AB_normalized = AB / AB_norm
    BC_normalized = BC / BC_norm
    
    # Calculate the dot product of the normalized vectors
    dot_product = np.dot(AB_normalized, BC_normalized)
    
    # Clip dot_product to avoid possible numerical errors
    dot_product = np.clip(dot_product, -1.0, 1.0)
    
    # Calculate the angle in radians and then convert to degrees
    angle_radians = np.arccos(dot_product)
    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point

    # Calculate the vectors
    ba = a - b
    bc = c - b

    # Calculate the cosine of the angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    # Convert the angle from radians to degrees
    return np.degrees(angle)

def calculate_midpoint(coord1, coord2):
    x = (coord1[0] + coord2[0]) / 2
    y = (coord1[1] + coord2[1]) / 2
    z = (coord1[2] + coord2[2]) / 2
    return np.array([x, y, z])

def calculate_angle_using_cross_product(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point

    # Calculate the vectors
    ba = a - b
    bc = c - b

    # Compute the cross product of the vectors
    cross_product = np.cross(ba, bc)

    # Compute the dot product
    dot_product = np.dot(ba, bc)

    # Calculate the angle using the cross and dot products
    angle = np.arctan2(np.linalg.norm(cross_product), dot_product)

    # Convert the angle from radians to degrees
    return np.degrees(angle)


# Function to calculate the distance between two points
def calculate_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a - b)
    
def calculate_angle_2d(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle

# Load the model
#model = YOLO('model_final.pt')
# Load the trained model


model = load_model('lstm_model.h5')

# Read and preprocess the CSV file to get the label encoder
data = pd.read_csv('csv/output.csv')
label_encoder = LabelEncoder()
#label_encoder = joblib.load('label_encoder.pkl')

data['class_name'] = label_encoder.fit_transform(data['class_name'])


# Standardize the features
scaler = StandardScaler()
scaler.fit(data[['right_shoulder_angle',
            'right_elbow_angle',
            'right_hip_angle',
            'right_knee_angle',
            'left_shoulder_angle',
            'left_elbow_angle',
            'left_hip_angle',
            'left_knee_angle']])

# Process the video
file_path = 'allpose.mp4'

cap = cv2.VideoCapture(file_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()


# Define the codec and create VideoWriter object
window_name = 'Frame'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 720,750 )  # Resize the window to 800x600

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3, min_tracking_confidence=0.3)


frame_count=0
ex_elbo_dis = [0.0, 0.0, 0.0]
lat_max = []
squat_angle = 1.0
squat_stand = 0.0
elbow_angle_temp =0.0
elbow_y = 0.0
puttext =0
error_el = 0

frame_data = []
predictions = []
sequence_length=20
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    pose_results = pose.process(frame_rgb)
  
    if puttext != 0 and error_el != 0:
        cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        puttext -= 1
        if puttext ==0:
            error_el = 0
            
    if pose_results.pose_landmarks:
    
        # Extract landmark positionst_ma
        landmarks = pose_results.pose_landmarks.landmark
    
        try:
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    
            right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
            left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]

            
            left_shoulder_x = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow_x = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist_x = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip_x = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                  landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee_x = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                   landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle_x = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            right_shoulder_x = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow_x = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist_x = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_knee_x = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle_x = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_hip_x = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    
    
    
            right_shoulder_angle = calculate_angle_2d(right_hip_x, right_shoulder_x, right_elbow_x)
            right_elbow_angle = calculate_angle_2d(right_shoulder_x, right_elbow_x, right_wrist_x)
            right_hip_angle = calculate_angle_2d(right_knee_x, right_hip_x, right_shoulder_x)
            right_knee_angle = calculate_angle_2d(right_hip_x, right_knee_x, right_ankle_x)
                            
            
            left_shoulder_angle = calculate_angle_2d(left_hip_x, left_shoulder_x, left_elbow_x)
            left_elbow_angle = calculate_angle_2d(left_shoulder_x, left_elbow_x, left_wrist_x)
            left_hip_angle = calculate_angle_2d(left_knee_x, left_hip_x, left_shoulder_x)
            left_knee_angle = calculate_angle_2d(left_hip_x, left_knee_x, left_ankle_x)
            
        except IndexError:
            right_shoulder_angle = np.nan
            right_elbow_angle = np.nan
            right_hip_angle = np.nan
            right_knee_angle = np.nan
            left_shoulder_angle = np.nan
            left_elbow_angle = np.nan
            left_hip_angle = np.nan
            left_knee_angle = np.nan
    else:
        right_shoulder_angle = np.nan
        right_elbow_angle = np.nan
        right_hip_angle = np.nan
        right_knee_angle = np.nan
        left_shoulder_angle = np.nan
        left_elbow_angle = np.nan
        left_hip_angle = np.nan
        left_knee_angle = np.nan
    
    frame_data.append([right_shoulder_angle,right_elbow_angle,right_hip_angle,
                       right_knee_angle,left_shoulder_angle,left_elbow_angle,left_hip_angle,left_knee_angle,])
    
    # Keep the latest sequence_length frames
    if len(frame_data) >= sequence_length:
        sequence = scaler.transform(frame_data[-sequence_length:])
        sequence = np.expand_dims(sequence, axis=0)
        prediction = model.predict(sequence)
        predicted_class = np.argmax(prediction, axis=1)
        predicted_label = label_encoder.inverse_transform(predicted_class)[0]
        predictions.append(predicted_label)
        
        
        
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        
        right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
        
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
        
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

        right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
        left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]

        names = label_encoder.classes_
        if predicted_label == names[0]: # bicep
            if left_shoulder.visibility > 0.5:
                h, w, _ = frame.shape
                midpoint_pixel = (int(left_shoulder.x * w), int(left_shoulder.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                
                angle = calculate_angle(
                [left_shoulder.x, left_shoulder.y, left_shoulder.z],
                [left_elbow.x, left_elbow.y, left_elbow.z],
                [left_wrist.x, left_wrist.y, left_wrist.z]
                )
                cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                if frame_count > 5:
                    elbow_angle = calculate_angle([left_shoulder.x, left_shoulder.y, left_shoulder.z],
                    [left_elbow.x, left_elbow.y, left_elbow.z],
                    [left_hip.x, left_hip.y, left_hip.z])
                    sho_hip_dis = calculate_distance(
                                [left_hip.x, left_hip.y, left_hip.z],
                                [left_shoulder.x, left_shoulder.y, left_shoulder.z]
                            )

                    if elbow_y >  left_elbow.x:
                        if abs(elbow_y-left_elbow.x) > sho_hip_dis*0.1 :
                            error_el = 1
                            puttext = 15 # 1 sec
                            cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    else :
                        if abs(left_elbow.x-elbow_y) > sho_hip_dis*0.1 :
                            error_el = 1
                            puttext = 15 # 1 sec
                            cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    

                if frame_count % 5 == 0: # 2 sec
                    elbow_angle_temp = calculate_angle([left_shoulder.x, left_shoulder.y, left_shoulder.z],
                    [left_elbow.x, left_elbow.y, left_elbow.z],
                    [left_hip.x, left_hip.y, left_hip.z])
                    elbow_y = left_elbow.x
            else:
                h, w, _ = frame.shape
                midpoint_pixel = (int(right_shoulder.x * w), int(right_shoulder.y * h))
                cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
                
                angle = calculate_angle(
                [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [right_elbow.x, right_elbow.y, right_elbow.z],
                [right_wrist.x, right_wrist.y, right_wrist.z]
                )
                cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                if frame_count > 5:
                    elbow_angle = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [right_elbow.x, right_elbow.y, right_elbow.z],
                    [right_hip.x, right_hip.y, right_hip.z])
                    if elbow_angle >  elbow_angle_temp:
                        if abs(elbow_angle-elbow_angle_temp)> 50 :
                            error_el = 1
                            puttext = 15 # 1 sec
                            cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    else :
                        if abs(elbow_angle_temp-elbow_angle)> 50 :
                            error_el = 1
                            puttext = 15 # 1 sec
                            cv2.putText(frame, f'FIX Elbow', (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                if frame_count % 5 == 0: # 2 sec
                    elbow_angle_temp = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                    [right_elbow.x, right_elbow.y, right_elbow.z],
                    [right_hip.x, right_hip.y, right_hip.z])
    
        elif predicted_label == names[3]: # dumbbell row
            leftangle = calculate_angle(
            [left_shoulder.x, left_shoulder.y, left_shoulder.z],
            [left_elbow.x, left_elbow.y, left_elbow.z],
            [left_wrist.x, left_wrist.y, left_wrist.z]
                )
            rightangle = calculate_angle(
            [right_shoulder.x, right_shoulder.y, right_shoulder.z],
            [right_elbow.x, right_elbow.y, right_elbow.z],
            [right_wrist.x, right_wrist.y, right_wrist.z]
                )
             
            if leftangle > rightangle:
                back_mid = calculate_midpoint([left_shoulder.x, left_shoulder.y, left_shoulder.z],[left_hip.x,left_hip.y,left_hip.z])
                #h, w, _ = frame.shape
                #midpoint_pixel = (int(back_mid[0] * w), int(back_mid[1] * h))
                #cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
    
                if left_elbow.y*1.05 < back_mid[1]:
                    cv2.putText(frame, f'Elbow down', (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
                if leftangle < np.degrees(1.5 ) : # 100 degree
                    cv2.putText(frame, f'keep left arm straight', (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else :
                back_mid = calculate_midpoint([right_shoulder.x, right_shoulder.y, right_shoulder.z],[right_hip.x,right_hip.y,right_hip.z])
                if right_elbow.y *1.05< back_mid[1]:
                    cv2.putText(frame, f'Elbow down', (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
                if rightangle < np.degrees(1.5 ) : 
                    cv2.putText(frame, f'keep right arm straight', (10, 290), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)                            
    
        elif predicted_label ==  names[1]:  # Incline Dumbbell Press
            angle = calculate_angle(
            [left_shoulder.x, left_shoulder.y, left_shoulder.z],
            [left_elbow.x, left_elbow.y, left_elbow.z],
            [left_wrist.x, left_wrist.y, left_wrist.z]
            )
            cv2.putText(frame, f'Elbow Angle: {int(angle)}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            h, w, _ = frame.shape
            midpoint_pixel = (int(right_shoulder.x * w), int(right_shoulder.y * h))
            cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
            midpoint_pixel = (int(right_elbow.x * w), int(right_elbow.y * h))
            cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
            midpoint_pixel = (int(right_hip.x * w), int(right_hip.y * h))
            cv2.circle(frame, midpoint_pixel, 5, (0, 255, 0), -1)  # Green dot
            #cv2.putText(frame, f'leftangle: {int(leftangle)}', (10, 230), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            
            if left_elbow.y*0.95 > left_shoulder.y:
                shoulder_angle = calculate_angle([right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [right_elbow.x, right_elbow.y, right_elbow.z],
                [right_hip.x, right_hip.y, right_hip.z])
                cv2.putText(frame, f'shoul_ang: {int(shoulder_angle)}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
                if shoulder_angle > np.degrees(2.7925268):
                    cv2.putText(frame, f'too wide elbow', (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
        elif predicted_label ==names[2]:  # Lat Pull Down
    
            shoulder_distance = calculate_distance(
                [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [left_shoulder.x, left_shoulder.y, left_shoulder.z]
            )
            wrist_distance = calculate_distance(
                [right_wrist.x, right_wrist.y, right_wrist.z],
                [left_wrist.x, left_wrist.y, left_wrist.z]
            )
            #cv2.putText(frame, f'Wrist Distance: {wrist_distance:.2f}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            #cv2.putText(frame, f'shoul Distance: { shoulder_distance * 2.2:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
            if wrist_distance > shoulder_distance * 2.2:
                cv2.putText(frame, "Keep Wrists Closer", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
        elif predicted_label == names[4]:  # Squat (already implemented)
            error_sh_kne = 0
            error_posi = 0
            
            shoulder_distance = calculate_distance(
                [right_shoulder.x, right_shoulder.y, right_shoulder.z],
                [left_shoulder.x, left_shoulder.y, left_shoulder.z]
            )
            knee_distance = calculate_distance(
                [right_knee.x, right_knee.y, right_knee.z],
                [left_knee.x, left_knee.y, left_knee.z]
            )
            ankle_distance = calculate_distance(
                [right_ankle.x, right_ankle.y, right_ankle.z],
                [left_ankle.x, left_ankle.y, left_ankle.z]
            )
            
            squat_angle = calculate_angle(
            [right_hip.x, right_hip.y, right_hip.z],
            [right_knee.x, right_knee.y, right_knee.z],
            [right_ankle.x, right_ankle.y, right_ankle.z]
            ) 
            #flip =0
            if right_knee.visibility > 0.5:
                if squat_angle > np.degrees(2.7925268 ) :
                    squat_stand = abs(right_knee.x-right_foot_index.x)
                    if ankle_distance > (shoulder_distance * 1.43) :
                        error_sh_kne = 1
                        cv2.putText(frame, f'adjust your ankle closer', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        
                    if ankle_distance < (shoulder_distance * 0.8):
                        error_sh_kne = 1
                        cv2.putText(frame, f'adjust your ankle wider', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                #else:
                    #if right_knee.x > right_foot_index.x :
                       #flip=1
                    
    
                if squat_stand !=0:
                    if right_knee.x < right_foot_index.x :
                        if right_knee.x + (squat_stand) < right_foot_index.x:
                            cv2.putText(frame, f'false position', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    #if flip== 1:
                        #if right_knee.x - (squat_stand) > right_foot_index.x:
                            #cv2.putText(frame, f'false position xxxx', (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                if squat_angle > np.degrees(2.7925268 ) :
                    squat_stand = abs(left_knee.x-left_foot_index.x)
                    if ankle_distance > (shoulder_distance * 1.43) :
                        error_sh_kne = 1
                        cv2.putText(frame, f'adjust your ankle closer', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
                    if ankle_distance < (shoulder_distance * 0.8):
                        error_sh_kne = 1
                        cv2.putText(frame, f'adjust your ankle wider', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2) 
                #else:
                    #if left_knee.x > left_foot_index.x :
                        #flip=1
                        
                if squat_stand !=0:
                    if left_knee.x < left_foot_index.x :
    
                        if left_knee.x + (squat_stand) < left_foot_index.x:
                            cv2.putText(frame, f'false position', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
        # Overlay prediction on the video frame
        cv2.putText(frame, f'Prediction: {predicted_label}', (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
    # Draw the skeleton on the frame
    mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    # Display the frame
    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when job is finished
cap.release()
# out.release()
cv2.destroyAllWindows()


In [5]:
from sklearn.preprocessing import LabelEncoder
import joblib


# Loading the LabelEncoder for inference
label_encoder = joblib.load('label_encoder.pkl')
print("Classes used during training:", label_encoder.classes_)

Classes used during training: ['bicep' 'dumbbell-row' 'incline' 'lat-pulldown' 'squat']


In [6]:
from sklearn.preprocessing import LabelEncoder
import joblib


# Loading the LabelEncoder for inference
label_encoder = joblib.load('test_label_encoder.pkl 2')
print("Classes used during training:", label_encoder.classes_)

Classes used during training: ['bicep' 'incline' 'lat' 'row' 'squat']
